In [ ]:
!sudo add-apt-repository ppa:saiarcot895/chromium-beta -y
!sudo apt remove chromium-browser
!sudo snap remove chromium
!sudo apt install chromium-browser

!pip3 install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
# TABLE PREMIER LEAGUE for each years in specific Country 
# driver = webdriver.Chrome()
countryList=['premier-league','ligue-1','laliga','serie-a','bundesliga']
countryLeague = ['https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id=',
'https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1/plus/?saison_id=',
'https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1/plus/?saison_id=',
'https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/plus/?saison_id=',
'https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=']

for index in range(len(countryLeague)):
    position,years,teamName,matches,total,pts,teamId,leagueName=[],[],[],[],[],[],[],[]
    for i in range(2015,2022):
        link = countryLeague[index]+str(i)
        driver.get(link)
        html = driver.page_source;
        reSoup = BeautifulSoup(html,'html.parser')
        reSoup.select('#yw4 > table > tbody')
        search=reSoup.select('#yw4 > table > tbody')[0].find_all('tr')
        for j in range(len(search)):
            years.append(int(i))
            leagueName.append(countryList[index])
            position.append(search[j].find_all("td")[0].text.strip())
            teamName.append(search[j].find_all("td")[2].a.get('title'))
            teamId.append(search[j].find_all("td")[2].a.get('href').split('/verein/')[1].split('/saison_id/')[0])
            matches.append(search[j].find_all("td")[3].text)
            total.append(search[j].find_all("td")[4].text)
            pts.append(search[j].find_all("td")[5].text)
    if(index==0):
        df=pd.DataFrame({'leagueName':leagueName,'season':years,'teamId':teamId,'teamName':teamName,'position':position,'matches':matches,'total':total,'pts':pts})
    else:
        df1= pd.DataFrame({'leagueName':leagueName,'season':years,'teamId':teamId,'teamName':teamName,'position':position,'matches':matches,'total':total,'pts':pts})
        df=pd.concat([df,df1],axis=0)
df.reset_index(drop=True)
df.to_parquet('AllTeamTablePosition.parquet',engine='pyarrow')
